In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import missingno as msno
from tqdm import tqdm

In [2]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
DATA_PATH = '/opt/ml/input/data/train_dataset/train_data.csv'
train_org_df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
train_org_df = train_org_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

CPU times: user 4.32 s, sys: 512 ms, total: 4.83 s
Wall time: 4.84 s


In [3]:
train_df = train_org_df.copy()

In [4]:
diff = train_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
diff = diff.fillna(pd.Timedelta(seconds=0))
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

train_df['elapsed'] = diff
train_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7.0
...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24.0
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,6632178.0
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11.0
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46.0


In [5]:
tmp = ""
idx = []

for i in tqdm(train_df.index):
    if tmp == train_df.iloc[i].testId:
        continue
    else:
        tmp = train_df.iloc[i].testId
        idx.append(i)

100%|██████████| 2266586/2266586 [08:44<00:00, 4318.62it/s]


In [6]:
train_df.loc[idx, "elapsed"] = 0

In [7]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
DATA_PATH = '/opt/ml/input/data/train_dataset/test_data.csv'
test_org_df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
test_org_df = test_org_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

CPU times: user 380 ms, sys: 24 ms, total: 404 ms
Wall time: 402 ms


In [8]:
test_df = test_org_df.copy()
test_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623


In [9]:
diff = test_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
diff = diff.fillna(pd.Timedelta(seconds=0))
diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

test_df['elapsed'] = diff
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.0
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,26.0
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,94.0
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5.0
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,7.0
...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,4721258.0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,18.0
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,21.0
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,89.0


In [10]:
tmp = ""
idx = []

for i in tqdm(test_df.index):
    if tmp == test_df.iloc[i].testId:
        continue
    else:
        tmp = test_df.iloc[i].testId
        idx.append(i)

100%|██████████| 260114/260114 [01:01<00:00, 4249.87it/s]


In [11]:
test_df.loc[idx, "elapsed"] = 0

In [12]:
train_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7.0
...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24.0
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0.0
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11.0
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46.0


In [13]:
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.0
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,26.0
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,94.0
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5.0
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,7.0
...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,18.0
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,21.0
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,89.0


In [14]:
all_df = pd.concat([train_df, test_df], sort=False)
all_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7.0
...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,18.0
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,21.0
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,89.0


In [15]:
def f(threshold):
    return sum(all_df.elapsed > threshold) / all_df.shape[0]

In [16]:
f(250)

0.04378002928721257

In [17]:
all_df.loc[(all_df.elapsed != 0) & (all_df.elapsed <= 250), "elapsed"].mean()

42.215718834173316

In [18]:
tmp_mean = all_df.loc[(all_df.elapsed != 0) & (all_df.elapsed <= 250), "elapsed"].mean()
tmp_mean

42.215718834173316

In [19]:
train_df.loc[train_df.elapsed == 0, "elapsed"] = tmp_mean

In [20]:
sum(train_df.elapsed == 0)

0

In [21]:
test_df.loc[test_df.elapsed == 0, "elapsed"] = tmp_mean

In [22]:
sum(test_df.elapsed == 0)

0

In [23]:
train_df.loc[train_df.elapsed > 250, "elapsed"] = tmp_mean

In [24]:
sum(train_df.elapsed > 250)

0

In [25]:
test_df.loc[test_df.elapsed > 250, "elapsed"] = tmp_mean

In [26]:
sum(test_df.elapsed > 250)

0

In [27]:
train_df.to_csv("/opt/ml/input/data/train_dataset/train_data_add_elapsed.csv", index=False)

In [28]:
test_df.to_csv("/opt/ml/input/data/train_dataset/test_data_add_elapsed.csv", index=False)

In [29]:
len(train_df.elapsed.unique())

251

In [30]:
train_df.elapsed.mean()

42.22447667722626

In [31]:
test_df.elapsed.mean()

42.1394045855978

In [32]:
all_df = pd.concat([train_df, test_df], sort=False)
all_df.elapsed.mean()

42.215718834173316